In [174]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from IPython.display import display, clear_output

In [180]:
pd.options.display.max_colwidth = 500
pd.options.display.max_columns = 500

In [55]:
LISTING = 'https://www.pilcomayo.net/calidaddeaguas'
PHPSESSID = '***************'

In [185]:
def make_indices():
    response = requests.get(LISTING)
    html = BeautifulSoup(response.text, 'html.parser')

    def make_indice(tabla):

        return pd.DataFrame([[tabla, a.get_text()] + re.findall('(?<=\=)[^\&$]*', a['data-vars']) for a in html.select(f'#{tabla} td a.modal-ajax')], columns=['tipo', 'lugar', 'rela_syspubl01', 'id_syscali01'])

    indice = pd.concat([make_indice(tipo) for tipo in ['mensuales', 'semestrales']])
    
    return indice

In [186]:
indice = make_indices()

In [187]:
indice

,tipo,lugar,rela_syspubl01,id_syscali01
0,mensuales,Tarapaya,16,5
1,mensuales,El Puente,9,3
2,mensuales,Palca Grande,5,4
3,mensuales,Colavi - Canutillos,63,668
4,mensuales,Tacobamba - Agua arriba confluencia Pilcomayo - Tacobamba,297,670
5,mensuales,Pilcomayo - Agua arriba confluencia Pilcomayo - Tacobamba,296,669
6,mensuales,Villamontes,6,2
7,mensuales,Misión La Paz,3,1
0,semestrales,Potosí - Naciente río La Ribera,41,16
1,semestrales,Liviara,26,27


In [81]:
def get_puntos():
    
    cookies = {
        'PHPSESSID': PHPSESSID,
    }

    data = {
        'rela_syspubl01': '272',
        'id_syscali01': '364',
    }

    response = requests.post('https://www.pilcomayo.net/modulos/reportes/calidad_aguas/php/ver_calidad_aguas_preview_v2.php', cookies=cookies, data=data)
    puntos = pd.DataFrame([re.findall('\'([^\']*)\'', punto) for punto in re.findall('mostrar_puntos_calidad_aguas\([^\)]*\)', response.text)], columns=['latitud', 'longitud', 'id_syscali01'])
    
    return puntos

In [79]:
puntos = get_puntos()

In [80]:
puntos

,latitud,longitud,id_syscali01
0,-22.3792,-62.5189,1
1,-21.2794,-63.4546,2
2,-21.2396,-65.2091,3
3,-20.7421,-65.2414,4
4,-19.4719,-65.7936,5
...,...,...,...
859,-21.2579,-63.5121,1296
860,-22.3781,-62.5186,1297
861,-25.2641,-57.7243,1298
862,-25.7884,-58.0215,1299


In [172]:
def get_punto(id_syscali01):
    
    cookies = {
        'PHPSESSID': PHPSESSID,
    }

    params = {
        'id_syscali01': id_syscali01,
    }

    response = requests.get('https://www.pilcomayo.net/modulos/reportes/calidad_aguas/php/ver_calidad_agua_reporte.php', params=params, cookies=cookies)
    html = BeautifulSoup(response.text, 'html.parser')
    return html

In [109]:
def unroll_table(table, columns):
    return [[field.get_text() for field in row.select('td')] for row in table.select('tr')[1:] if len(row.select('td')) == columns]

In [134]:
def parse_tabla_punto(t):
    data = {}
    for row in t:
        data[row[0].strip()] = row[1]
        data[row[2].strip()] = row[3]
    return data

In [170]:
def parse_lugar(html):
    table = html.select('table')[0]
    t = unroll_table(table, 4)
    return parse_tabla_punto(t)

In [163]:
def parse_tabla_mediciones(t):
    data = {}
    for row in t:
        data[f'{row[0].strip()} ({row[1].strip()})'] = row[2].strip()
    return data

In [158]:
def parse_mediciones(table):
    t = unroll_table(table, 4)[1:]
    return parse_tabla_mediciones(t)

In [160]:
def parse_titulo_mediciones(table):
    return table.select('th')[0].get_text()

In [169]:
def parse_punto(html):
    mediciones = parse_lugar(html)
    for table in html.select('table')[1:]:
        mediciones.update(parse_mediciones(table))
    return mediciones

In [183]:
data = []

In [184]:
for i, punto in puntos.iterrows():
    clear_output(wait=True)
    print(i)
    html = get_punto(punto['id_syscali01'])
    mediciones = parse_punto(html)
    data.append({**punto.to_dict(), **mediciones})

863


In [189]:
df = pd.DataFrame(data)

In [194]:
df

,latitud,longitud,id_syscali01,Latitud:,Longitud:,Río:,Altura,Fecha de Muestreo,Hora de Muestreo,Velocidad media,Caudal,Campaña,Responsable,Conductividad específica (T ref. 25°C) (uS/cm),Oxígeno disuelto (mg/l O2),pH (u pH),Sólidos disueltos totales (mg/l),Sólidos suspendidos totales (mg/l),Sólidos totales (mg/l),Temperatura (C),Calcio disuelto (mg/l Ca),Calcio total (mg/l Ca),Dureza total (mg/l CaCO3),Magnesio disuelto (mg/l Mg),Magnesio total (mg/l Mg),Potasio disuelto (mg/l K),Potasio total (mg/l K),Sodio disuelto (mg/l Na),Sodio total (mg/l Na),Demanda bioquímica de oxígeno (mg/l O2),Demanda química de oxígeno (mg/l O2),Arsenico disuelto (ug/l As),Arsenico total (ug/l As),Cadmio disuelto (ug/l Cd),Cadmio total (ug/l Cd),Cobre disuelto (ug/l Cu),Cobre total (ug/l Cu),Cromo disuelto (ug/l Cr),Cromo total (ug/l Cr),Hierro disuelto (ug/l Fe),Hierro total (ug/l Fe),Manganeso disuelto (ug/l Mn),Manganeso total (ug/l Mn),Mercurio disuelto (ug/l Hg),Mercurio total (ug/l Hg),Niquel disuelto (ug/l Ni),Niquel total (ug/l Ni),Plata disuelto (ug/l Ag),Plata total (ug/l Ag),Plomo disuelto (ug/l Pb),Plomo total (ug/l Pb),Zinc disuelto (ug/l Zn),Zinc total (ug/l Zn),Arsenico suspensión (mg/kg As),Cadmio suspensión (mg/kg Cd),Cobre suspensión (mg/kg Cu),Cromo suspensión (mg/kg Cr),Hierro suspensión (mg/kg Fe),Manganeso suspensión (mg/kg Mn),Niquel suspensión (mg/kg Ni),Plomo suspensión (mg/kg Pb),Zinc en suspensión (mg/kg Zn),Turbiedad (UNT),Plata suspensión (mg/kg Ag),Mercurio suspensión (mg/kg Hg),pH (mV) (mV),Salinidad (Sal),Saturación de oxígeno (%),Alcalinidad total (mg/l CaCO3),Cloruros (mg/l Cl -),Sulfatos (mg/l SO4=),Fósforo disuelto (mg/l PO4),Fósforo total (mg/l PO4),Nitrato (mg/l NO3),Nitrito (mg/l NO2),Nitrógeno amoniacal (mg/l NH4),Nitrógeno total Kjeldahl (mg/l N),Carbono organico disuelto (mg/l),Coliformes fecales (en NMP) (NMP/100 ml),Coliformes totales (en NMP) (NMP/100 ml),Cianuros libre (mg/l CN-),Fenoles (mg/l C6H6O),Sulfuros (mg/l S=),Aerobios mesófilos totales (en UFC) (UFC/100 ml),Coliformes fecales (en UFC) (UFC/100 ml),Coliformes totales (en UFC) (UFC/100 ml),Cianuros libre (CN-),Sulfuros (S=),Cianuros (mg/l CN-),Color (u PtCo),Boro disuelto (ug/l B),Boro total (ug/l B),Selenio disuelto (ug/l Se),Selenio total (ug/l Se),Hidrocarburos totales (mg/l),Boro suspensión (mg/kg B),Selenio suspensión (mg/kg Se),Cobre total (Cu),Cromo disuelto (Cr),Cromo total (Cr),Hierro disuelto (Fe),Hierro total (Fe),Manganeso disuelto (Mn),Manganeso total (Mn),Mercurio disuelto (Hg),Mercurio total (Hg),Niquel disuelto (Ni),Niquel total (Ni),Plata disuelto (Ag),Plata total (Ag),Plomo disuelto (Pb),Plomo total (Pb),Selenio disuelto (Se),Selenio total (Se),Zinc disuelto (Zn),Zinc total (Zn),Fenoles (C6H6O),Aerobios mesófilos totales (en UFC) (UFC/ml),Hidrocarburos totales (),Cadmio total (Cd),Cobre disuelto (Cu),Fósforo total (PO4),Arsenico total (As),Boro disuelto (B),Boro total (B),Cadmio disuelto (Cd),Demanda bioquímica de oxígeno (O2),Demanda química de oxígeno (O2),Arsenico disuelto (As),Magnesio total (Mg),Potasio disuelto (K),Sodio disuelto (Na),Sulfatos (SO4=),Nitrato (NO3),Nitrito (NO2),Nitrógeno amoniacal (NH4),Fósforo disuelto (PO4),Alcalinidad Fenolftaleina (mg/l CaCO3),Turbiedad (NTU),Bismuto disuelto (ug/l Bi),Bismuto total (ug/l Bi),Talio disuelto (ug/l Tl),Talio total (ug/l Tl),Bismuto suspensión (mg/kg Bi),Talio en suspensión (mg/kg Tl),Presión Parcial (mm Hg),Resistividad (Ohm.cm),Bismuto total (Bi),Talio disuelto (Tl),Talio total (Tl),Bismuto disuelto (Bi),Aerobios mesófilos totales (en NMP) (NMP/100 ml),Cianuros (CN-),Coliformes totales (en UFC) (),Coliformes totales (en NMP) (),Alcalinidad total (CaCO3),Dureza total (CaCO3),Magnesio disuelto (Mg),Potasio total (K),Sodio total (Na),Nitrógeno total Kjeldahl (N),Aerobios mesófilos totales (en UFC) (),Coliformes fecales (en NMP) (),Coliformes fecales (en UFC) (),pH en laboratorio (u pH),Sólidos sedimentables (ml/l),Calcio disuelto (Ca),Calcio total (Ca),Cloruros (Cl -),Car

In [195]:
df.to_csv('data/mediciones.csv', index=False)